In [1]:
from pymc3 import Beta
import numpy as np
rand = np.random.rand
import scipy.stats as stats
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
import scipy.stats as stats
class Bandits(object):
    """
    This class represents N bandits machines.
    parameters:
        p_array: a (n,) Numpy array of probabilities >0, <1.
    methods:
        pull( i ): return the results, 0 or 1, of pulling 
                   the ith bandit.
    """
    def __init__(self, p_array):
        self.p = p_array
        self.optimal = np.argmax(p_array)
        
    def pull( self, i ):
        #i is which arm to pull
        return rand() < self.p[i]
    
    def __len__(self):
        return len(self.p)

    
class BayesianStrategy( object ):
    """
    Implements a online, learning strategy to solve
    the Multi-Armed Bandit problem.
    
    parameters:
        bandits: a Bandit class with .pull method
    
    methods:
        sample_bandits(n): sample and train on n pulls.
    attributes:
        N: the cumulative number of samples
        choices: the historical choices as a (N,) array
        bb_score: the historical score as a (N,) array
    """
    
    def __init__(self, bandits):
        
        self.bandits = bandits
        self.n_bandits = len( self.bandits )
        self.wins = np.zeros( self.n_bandits )
        self.trials = np.zeros(self.n_bandits )
        self.N = 0
        self.choices = []
        self.bb_score = []

    
    def sample_bandits( self, n=1 ):
        
        bb_score = np.zeros( n )
        choices = np.zeros( n )
        
        for k in range(n):
            #sample from the bandits's priors, and select the largest sample
            choice = np.argmax( pm.Beta.dist( 1 + self.wins, 1 + self.trials - self.wins).random(size=self.n_bandits) )
            
            #sample the chosen bandit
            result = self.bandits.pull( choice )
            
            #update priors and score
            self.wins[ choice ] += result
            self.trials[ choice ] += 1
            bb_score[ k ] = result 
            self.N += 1
            choices[ k ] = choice
            
        self.bb_score = np.r_[ self.bb_score, bb_score ]
        self.choices = np.r_[ self.choices, choices ]
        return 

In [ ]:
    figsize(11.0, 10)

beta = stats.beta
x = np.linspace(0.001, .999, 200)



def plot_priors(bayesian_strategy, prob, lw=3, alpha=0.2, plt_vlines=True):
    # plotting function
    wins = bayesian_strategy.wins
    trials = bayesian_strategy.trials
    for i in range(prob.shape[0]):
        y = beta(1 + wins[i], 1 + trials[i] - wins[i])
        p = plt.plot(x, y.pdf(x), lw=lw)
        c = p[0].get_markeredgecolor()
        plt.fill_between(x, y.pdf(x), 0, color=c, alpha=alpha,
                         label="underlying probability: %.2f" % prob[i])
        if plt_vlines:
            plt.vlines(prob[i], 0, y.pdf(prob[i]),
                       colors=c, linestyles="--", lw=2)
        plt.autoscale(tight="True")
        plt.title("Posteriors After %d pull" % bayesian_strategy.N +
                  "s" * (bayesian_strategy.N > 1))
        plt.autoscale(tight=True)
    return


In [ ]:
import pymc3 as pm
hidden_prob = np.array([0.5,  0.75])
bandits = Bandits(hidden_prob)
bayesian_strat = BayesianStrategy(bandits)

draw_samples = [1, 1, 3, 10, 10, 25, 50, 100, 200, 600]

for j, i in enumerate(draw_samples):
    plt.subplot(5, 2, j + 1)
    bayesian_strat.sample_bandits(i)
    plot_priors(bayesian_strat, hidden_prob)
    # plt.legend()
    plt.autoscale(tight=True)
plt.tight_layout()

In [ ]:
import theano.tensor as tt
class BanditsGaussian(object):
    """
    This class represents N bandits machines.
    parameters:
        p_array: a (n,) Numpy array of probabilities >0, <1.
    methods:
        pull( i ): return the results, 0 or 1, of pulling 
                   the ith bandit.
    """
    def __init__(self, mu_array):
        self.mu = mu_array
        self.optimal = np.argmax(mu_array)
        dist_list = []
#         for i in range(len(mu_array)):
#             dist_list.append(pm.TruncatedNormal.dist(mu=mu_array[i],sd=1,lower=0,upper = 1))
            
        self.dist_list = pm.TruncatedNormal.dist(mu=mu_array,sd=1,lower=0,upper = 1)
        
        
    def pull( self, i ):
        #i is which arm to pull
        return self.dist_list.random()[i]
        
        
    def __len__(self):
        return len(self.dist_list)

In [ ]:
bandit = BanditsGaussian([0.5,0.6])
print(bandit.pull(0))

In [ ]:
def from_posterior(param, samples):
    smin, smax = np.min(samples), np.max(samples)
    width = smax - smin
    x = np.linspace(smin, smax, 100)
    y = stats.gaussian_kde(samples)(x)

    # what was never sampled should have a small probability but not 0,
    # so we'll extend the domain and use linear approximation of density on it
    x = np.concatenate([[x[0] - 3 * width], x, [x[-1] + 3 * width]])
    y = np.concatenate([[0], y, [0]])
    return Interpolated(param, x, y)



In [ ]:
import pymc3 as pm
import numpy as np
basic_model = pm.Model()

with basic_model:

    # Priors for unknown model parameters
    mu1 = pm.Uniform(name = "mu1",lower = 0,upper = 1)
    mu2 = pm.Uniform(name = "mu2",lower = 0,upper = 1)
    bandit1_data = np.array([])

    bandit2_data = np.array([])
    bandit1_obs = pm.TruncatedNormal(name="bandit1",mu = mu1,sigma=1,lower=0,upper=1,observed = bandit1_data)
    bandit2_obs = pm.TruncatedNormal(name="bandit2",mu = mu1,sigma=1,lower=0,upper=1,observed = bandit2_data)
    


    # draw 1000 posterior samples
    trace = pm.sample(1)



In [ ]:
traces = [trace]
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
print("Posterior distributions after " + str(len(traces)) + " iterations.")
cmap = mpl.cm.autumn
for param in ["mu1", "mu2"]:
    plt.figure(figsize=(8, 2))
    for update_i, trace in enumerate(traces):
        samples = trace[param]
        smin, smax = np.min(samples), np.max(samples)
        x = np.linspace(smin, smax, 100)
        y = stats.gaussian_kde(samples)(x)
        plt.plot(x, y, color=cmap(1 - update_i / len(traces)))
#     plt.axvline({"alpha": alpha_true, "beta0": beta0_true, "beta1": beta1_true}[param], c="k")
#     plt.ylabel("Frequency")
    plt.title(param)

plt.tight_layout();



In [ ]:
bandit_1_data = [bandit.pull(0) for i in range(100)]
bandit_2_data = [bandit.pull(1) for i in range (100)]
with pm.Model() as model:
    mu1 = pm.Uniform(name = "Bandit Mean 1",lower = 0,upper = 1)
    mu2 = pm.Uniform(name = "Bandit Mean 2",lower = 0,upper = 1)
    bandit1 = pm.TruncatedNormal(name="bandit1",mu = mu1,sigma=1,lower=0,upper=1,observed = bandit_1_data)
    bandit2 = pm.TruncatedNormal(name="bandit2",mu = mu2,sigma=1,lower=0,upper=1,observed = bandit_2_data)
    
    start = pm.find_MAP()
    trace = pm.sample(2000,  startt=start)

In [ ]:
import arviz as az

az.plot_trace(trace)